# Rate Table Matching
## Course Specific Fees for FRCC Courses

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
from pathlib import Path
import shutil

In [2]:
# Get current year
current_year = datetime.now().year % 100

FY_OPTIONS = [f"{(current_year - i) % 100:02d}" for i in range(4, -3, -1)]

# Allowed terms with friendly parsing
TERM_OPTIONS = ['SUMMER', 'FALL', 'SPRING']

print("Select Fiscal Year (FY):", FY_OPTIONS)
FY = input("Enter FY (e.g., 26): ").strip()

if FY not in FY_OPTIONS:
    raise ValueError(f"Invalid FY. Must be one of {FY_OPTIONS}")

print("\nSelect Term:", TERM_OPTIONS)
Term = input("Enter Term (Summer, Fall, Spring): ").strip().upper()

if Term not in TERM_OPTIONS:
    raise ValueError(f"Invalid Term. Must be one of {TERM_OPTIONS}")

print(f"\nUsing FY={FY}, Term={Term}")


Select Fiscal Year (FY): ['21', '22', '23', '24', '25', '26', '27']
Enter FY (e.g., 26): 26

Select Term: ['SUMMER', 'FALL', 'SPRING']
Enter Term (Summer, Fall, Spring): fall

Using FY=26, Term=FALL


In [3]:
MED_HIGH_RATES = {
    '20': 7.85,
    '21': 8.05,
    '22': 8.25,
    '23': 8.45,
    '24': 8.65,
    '25': 8.85,
    '26': 9.05,
    '27': 10.50
                }


FYFeeAmt = MED_HIGH_RATES.get(FY)

if FYFeeAmt is None:
    raise ValueError(f"Invalid FY '{FY}'. Must be one of {list(MED_HIGH_RATES.keys())}.")

print(f"Using FY={FY}, Fee={FYFeeAmt}")

TERM_MAP = {
    "SUMMER": {
        "Dig_content_fee": "A394",
        "Not_DCF": "A385",
        "Semester": "10"
    },
    "FALL": {
        "Dig_content_fee": "A392",
        "Not_DCF": "A383",
        "Semester": "20"
    },
    "SPRING": {
        "Dig_content_fee": "A393",
        "Not_DCF": "A384",
        "Semester": "30"
    }
}

term_key = Term.strip().upper()
if term_key in TERM_MAP:
    Dig_content_fee = TERM_MAP[term_key]["Dig_content_fee"]
    Not_DCF = TERM_MAP[term_key]["Not_DCF"]
    Semester = TERM_MAP[term_key]["Semester"]
else:
    Dig_content_fee = Not_DCF = Semester = "Please input a valid term"

print(f"Digital Content Fee Detail Code: {Dig_content_fee}\nAll other Detail Codes: {Not_DCF}")


Using FY=26, Fee=9.05
Digital Content Fee Detail Code: A392
All other Detail Codes: A383


In [13]:
# Function to get the current date for file renaming.
def get_current_date():
    current_date = datetime.now()
    formatted_date = current_date.strftime("%m-%d-%y")
    return formatted_date

current_date = get_current_date()
csv_doc = current_date + '.csv'
xlsx_doc = '_' + current_date + '.xlsx'


# Function to clean column names
def clean_column_names(df):

    # Remove leading and trailing quotation marks if they exist in column names
    df.columns = df.columns.str.strip('"')

    # Remove any non-alphanumeric characters at the start and end, and replace inner non-alphanumeric with underscores
    # Necessary to remove the ♀ from the first column
    df.columns = df.columns.str.replace(r'^[^\w]*|[^\w]*$', '', regex=True).str.replace(r'[^\w]+', '_', regex=True)

In [19]:
BASE_DIR   = Path.cwd().resolve()
DATA_DIR   = BASE_DIR / "Uploads"
OUTPUT_DIR = BASE_DIR / "Outputs"

DATA_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_DIR  :", DATA_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("Uploads currently present:", [p.name for p in DATA_DIR.iterdir()])

print("Drop files here in Jupyter:", DATA_DIR)
print("Currently present:", [p.name for p in DATA_DIR.iterdir()])

#output file names
Cleaned_CFL = f'Cleaned_CFL{xlsx_doc}'
Cleaned_CSF = f'FY{FY} Cleaned Course Specific Fees{xlsx_doc}'
Rate_Table = f'20{FY}{Semester} Rate Table{xlsx_doc}'

DATA_DIR  : /content/Uploads
OUTPUT_DIR: /content/Outputs
Uploads currently present: ['FY26 Course Specific Fees.xlsx', 'gokoutp.csv']
Drop files here in Jupyter: /content/Uploads
Currently present: ['FY26 Course Specific Fees.xlsx', 'gokoutp.csv']


In [6]:
def capture_upload(label: str, key: str, *, exts=(".xlsx",), canonical_name: str | None = None) -> Path:
    """
    Prompt user to upload a file for a specific role, then copy to a canonical name.
    If the uploaded file already has the canonical name, skip the copy.
    """
    before = {p.resolve() for p in DATA_DIR.iterdir()}

    print(f"\nUpload: {label}")
    print(f"   Accepted types: {', '.join(exts)}")
    print(f"   Drag the file into: {DATA_DIR}")
    input("   Press Enter after the upload completes... ")

    after = {p.resolve() for p in DATA_DIR.iterdir()}
    new_files = [Path(p) for p in (after - before) if Path(p).suffix.lower() in exts]

    if not new_files:
        candidates = [p for p in DATA_DIR.iterdir() if p.suffix.lower() in exts]
        if not candidates:
            raise FileNotFoundError(f"No {exts} files found in {DATA_DIR}.")
        print("\nNo new file detected. Pick one of the existing files:")
        for i, p in enumerate(candidates, 1):
            print(f"  {i}) {p.name}")
        idx = int(input("Enter number: ").strip())
        src = candidates[idx - 1]
    elif len(new_files) == 1:
        src = new_files[0]
    else:
        print("\nMultiple new files detected. Pick one:")
        picks = sorted(new_files)
        for i, p in enumerate(picks, 1):
            print(f"  {i}) {p.name}")
        idx = int(input("Enter number: ").strip())
        src = picks[idx - 1]

    # Canonical destination
    if canonical_name is None:
        canonical_name = f"{key}{src.suffix.lower()}"
    dst = DATA_DIR / canonical_name

    # If the source is already named as the canonical file, just use it
    try:
        if src.resolve() == dst.resolve():
            print(f"Using existing canonical file: {dst.name}")
            return dst
    except Exception:
        # On some filesystems resolve() might fail; fall back to string compare
        if str(src) == str(dst):
            print(f"Using existing canonical file: {dst.name}")
            return dst

    # Otherwise copy (overwrites if dst exists and is different)
    shutil.copy2(src, dst)
    print(f"Using: {src.name}  →  {dst.name}")
    return dst


In [7]:
csf_path = capture_upload(
    label=f"Course Specific Fees for FY{FY}",
    key="course_specific_fees",
    exts=(".xlsx",),
    canonical_name=f"FY{FY} Course Specific Fees.xlsx"
)


Upload: Course Specific Fees for FY26
   Accepted types: .xlsx
   Drag the file into: /content/Uploads
   Press Enter after the upload completes... 
Using existing canonical file: FY26 Course Specific Fees.xlsx


In [8]:
# 2) Course Fee Listing (CSV) — canonicalize to gokoutp.csv
cfl_path = capture_upload(
    label="Course Fee Listing (gokoutp.csv)",
    key="course_fee_listing",
    exts=(".csv",),
    canonical_name="gokoutp.csv"
)

print("Course Fee Listing saved as:", cfl_path)



Upload: Course Fee Listing (gokoutp.csv)
   Accepted types: .csv
   Drag the file into: /content/Uploads
   Press Enter after the upload completes... 
Using existing canonical file: gokoutp.csv
Course Fee Listing saved as: /content/Uploads/gokoutp.csv


In [9]:
BANNER_CourseFeeListing = DATA_DIR / "gokoutp.csv"
FRCC_CourseSpecificFees = DATA_DIR / f"FY{FY} Course Specific Fees.xlsx"

### Course Fee Listing manipulations.

In [28]:
df_Banner_CFL = pd.read_csv(BANNER_CourseFeeListing, delimiter=',', quotechar='"')

clean_column_names(df_Banner_CFL)

# Creating a Dictionary to rename columns into something more human friendly
rename_columns = {
    'SSBSECT_TERM_CODE': 'SEMESTER',
    'SSBSECT_CRN': 'CRN',
    'SSBSECT_SUBJ_CODE': 'SUBJECT',
    'SSBSECT_CRSE_NUMB': 'COURSE NUMBER',
    'SSBSECT_SEQ_NUMB': 'SECTION',
    'SSBSECT_CAMP_CODE': 'CAMPUS',
    'SSRATTR_ATTR_CODE': 'ATTR',
    'SSRFEES_DETL_CODE': 'DET CODE',
    'SSRFEES_AMOUNT': 'AMOUNT',
    'SSRFEES_FTYP_CODE': 'FEE TYPE (OLD)'
                }

# Renaming the columns
df_Banner_CFL.rename(columns=rename_columns, inplace=True, errors = 'ignore')

# Drop columns
df_Banner_CFL.drop(['SSBSECT_VPDI_CODE','SSBSECT_CREDIT_HRS', 'SSBSECT_BILL_HRS', 'SSBSECT_ENRL', 'SSBSECT_WAIT_COUNT', 'SSBSECT_LAB_HR', 'SSBSECT_LEC_HR', 'SSBSECT_OTH_HR', 'SSBSECT_PRNT_IND', 'SSBSECT_PTRM_CODE', 'SSBSECT_ACTIVITY_DATE', 'SSBSECT_PTRM_START_DATE', 'SSBSECT_PTRM_END_DATE', 'SSBSECT_CENSUS_ENRL_DATE', 'SSRATTR_ACTIVITY_DATE', 'SSRFEES_FEE_IND', 'SSRFEES_LEVL_CODE', 'SSBOVRR_COLL_CODE', 'SSBOVRR_DEPT_CODE', 'SSBOVRR_DIVS_CODE', 'SSBOVRR_TOPS_CODE', 'SSRMEET_BLDG_CODE', 'SSRMEET_START_DATE', 'SSRMEET_END_DATE', 'SSRMEET_BEGIN_TIME', 'SSRMEET_END_TIME', 'SSRMEET_HRS_WEEK', 'SSRMEET_ROOM_CODE', 'SSRMEET_CATAGORY', 'SSRMEET_SUN_DAY', 'SSRMEET_MON_DAY', 'SSRMEET_TUE_DAY', 'SSRMEET_WED_DAY', 'SSRMEET_THU_DAY', 'SSRMEET_FRI_DAY', 'SSRMEET_SAT_DAY'], axis=1, inplace=True)


''' Some data manipulation to refine out output and ensure we catch CONC attributes and drop duplicates'''
# Function to remove any value from Attribute column that isn't CONC
def conc_only(value):
    if value != "CONC":
        return ""
    else:
        return "CONC"

# Creating a dataset only where CRN's have a CONC attribute
CONC_df_CFL = df_Banner_CFL[df_Banner_CFL['ATTR'] == 'CONC']

# Creating a dateset only where there is a unique value in amount.
Unique_AMT_df_CFL = df_Banner_CFL.dropna(subset=['AMOUNT']).drop_duplicates(subset=['CRN', 'AMOUNT'])
Unique_AMT_df_CFL['ATTR'] = Unique_AMT_df_CFL['ATTR'].apply(conc_only)

# Creating a dataset where amount is nan
na_amt_df_CFL = df_Banner_CFL[df_Banner_CFL['AMOUNT'].isna()].drop_duplicates(subset=['CRN', 'ATTR'])
na_amt_df_CFL['ATTR'] = na_amt_df_CFL['ATTR'].apply(conc_only)

# Concatenating the three created datasets above to drop duplicates the required way.
df_Banner_CFL = pd.concat([CONC_df_CFL, Unique_AMT_df_CFL, na_amt_df_CFL])

df_Banner_CFL['ATTR2'] = df_Banner_CFL['ATTR'].apply(lambda x: 'MISSING' if pd.isna(x) or x == '' else x)
df_Banner_CFL['AMOUNT2'] = df_Banner_CFL['AMOUNT'].fillna('MISSING')

df_Banner_CFL['ATTR'] = df_Banner_CFL['ATTR'].apply(conc_only)

df_Banner_CFL.sort_values(by=['CRN', 'ATTR'], ascending = [True, False], inplace=True)

df_Banner_CFL.to_excel(OUTPUT_DIR / 'b4Drops.xlsx', index=False)

# Dropping duplicate values to ensure we are keeping any CONC that may have been dropped before this proccess
df_Banner_CFL = df_Banner_CFL.drop_duplicates(subset=['CRN', 'AMOUNT2'], keep='first')
df_Banner_CFL = df_Banner_CFL.drop(columns=['ATTR2', 'AMOUNT2'])

df_Banner_CFL.to_excel(OUTPUT_DIR / 'afterDrops.xlsx', index=False)

df_Banner_CFL['SECTION'] = df_Banner_CFL['SECTION'].astype(str).str.zfill(3)

# Drop rows where "CAMPUS" is FCX, FCW, FCZ, or FZZ
df_Banner_CFL = df_Banner_CFL[~df_Banner_CFL['CAMPUS'].str.contains('FCX|FCW|FCZ|FZZ', na=False)]

# Drop rows where Section is High School (37X, 38X, 39X, or 78X), Campus is FWO or FWC, and the Attribute is Concurrent (CONC) ## Added the other Campuses to match them
df_Banner_CFL = df_Banner_CFL[~((df_Banner_CFL['SECTION'].str.contains(r'37[A-Z]|28[A-Z]|37[A-Z]|38[A-Z]|39[A-Z]|78[A-Z]', na=False)) & ~((df_Banner_CFL['CAMPUS'].isin(['FWO', 'FWC', 'FLO', 'FLC', 'FBO', 'FBC'])) & (df_Banner_CFL['ATTR'] == 'CONC')))]

# Function to find HS courses and normalize them to all end in X or XX, have all else just end in XX except ALL
def modify_for_matching(value):
    if value == 'ALL':
        return 'ALL'
    else:
        section_str = str(value)
        if len(section_str) > 1 and section_str[:2] in ['37', '38', '39']:
            return section_str[:2] + 'X'
        else:
            return section_str[0] + 'XX'

df_Banner_CFL['MODIFIED_SECTION'] = df_Banner_CFL['SECTION'].apply(modify_for_matching)

# Function to remove any value from Attribute column that isn't CONC
def conc_only(value):
    if value != "CONC":
        return ""
    else:
        return "CONC"

df_Banner_CFL['ATTR'] = df_Banner_CFL['ATTR'].apply(conc_only)

# Sort values by SUBJECT then COURSE NUMBER
df_Banner_CFL.sort_values(by=['SUBJECT', 'COURSE NUMBER'], ascending = [True, True], inplace=True)

print("\nInformation about the transformed Course Fee Listing Dataset\n")
display(df_Banner_CFL.head())
print(f"\nColumns: {df_Banner_CFL.shape[1]} \nRows: {df_Banner_CFL.shape[0]}")
df_Banner_CFL.to_excel(OUTPUT_DIR / Cleaned_CFL, index=False)


Information about the transformed Course Fee Listing Dataset



,SEMESTER,CRN,SUBJECT,COURSE NUMBER,SECTION,CAMPUS,ATTR,DET CODE,AMOUNT,FEE TYPE (OLD),MODIFIED_SECTION
4661,202620,51421,AAA,50,0G3,FWC,,NaN,NaN,NaN,0XX
4743,202620,51445,AAA,50,0G2,FWC,,NaN,NaN,NaN,0XX
5392,202620,51655,AAA,50,0G5,FWC,,NaN,NaN,NaN,0XX
5855,202620,51803,AAA,50,0G4,FWC,,NaN,NaN,NaN,0XX
8045,202620,52434,AAA,50,0G1,FWC,,NaN,NaN,NaN,0XX



Columns: 11 
Rows: 3721


### Course Specific Fees manipulation

In [27]:
# Loading in the dataset so Python can manipulate it.
df_CSF = pd.read_excel(DATA_DIR / FRCC_CourseSpecificFees)

# Data Cleansing/Convert and Standardize the Data
df_CSF.columns = df_CSF.columns.str.upper().str.strip()

# Remove any white space from the string valueS in the columns below
df_CSF['CAMPUS'] = df_CSF['CAMPUS'].astype(str).str.strip()
df_CSF['SUBJECT'] = df_CSF['SUBJECT'].astype(str).str.strip()
df_CSF['COURSE NUMBER'] = df_CSF['COURSE NUMBER'].astype(str).str.strip()
df_CSF['SECTION'] = df_CSF['SECTION'].astype(str).str.strip()
df_CSF['FREQUENCY'] = df_CSF['FREQUENCY'].astype(str).str.strip()
df_CSF['EXPLANATION'] = df_CSF['EXPLANATION'].astype(str).str.strip()

# Replace variations of "ALL" with the actual word "ALL" for later assignments
df_CSF['CAMPUS'] = df_CSF['CAMPUS'].replace(['All', 'ALL', '', 'nan', 'NAN', np.nan], 'ALL')
df_CSF['COURSE NUMBER'] = df_CSF['COURSE NUMBER'].replace(['All', 'ALL', '', ' ', 'nan', 'NAN', np.nan], 'ALL')
df_CSF['SECTION'] = df_CSF['SECTION'].replace(['All', 'ALL', '', 'nan', 'NAN', np.nan], 'ALL')

# Forcing the column "Course Number" in the CSF data to be an integer. This will prevent having to transform the data from text (str) to numbers (int) in the actual excel document.
def text_num(CourseNumber):
    try:
        return int(CourseNumber)
    except (ValueError, TypeError):
        return CourseNumber

df_CSF['COURSE NUMBER'] = df_CSF['COURSE NUMBER'].apply(text_num)

# Utilize regex matching to remove any hyphens at the end of a string in "SECTION" with nothing.
df_CSF['SECTION'] = df_CSF['SECTION'].str.replace(r'-$', '', regex = True)

# Shorten the "EXPLANATION" column to a max of 30 characters.
df_CSF['EXPLANATION'] = df_CSF['EXPLANATION'].str.slice(0,30)

''' These functions below are to standardize the data, so each cell only has one value stored within for easier data manipulation'''
# Functions to split and expand rows by section so each entry is on it's own row.
def expand_rows(row):
    section = "ALL" if pd.isna(row['SECTION']) else str(row['SECTION']).strip()
    if section:
        sections = section.replace(' ', ',').split(',')
    else:
        sections = ["ALL"]

    new_rows = []
    for sec in sections:
        if sec:
            new_row = row.copy()
            new_row['SECTION'] = sec
            new_rows.append(new_row)
    return new_rows

# Function to expand rows so each campus gets its own row.
def expand_campuses(row):
    campus = str(row['CAMPUS']).strip() if pd.notna(row['CAMPUS']) else "ALL"
    campuses = [c.strip() for c in campus.split('/')]

    new_rows = []
    for camp in campuses:
        if camp:
            new_row = row.copy()
            new_row['CAMPUS'] = camp
            new_rows.append(new_row)
    return new_rows

# Applying the functions to the dataset

# Has to be a new variable, since expanded_rows returns a Series.
expanded_rows = df_CSF.apply(expand_rows, axis = 1)
df_CSF = pd.DataFrame([item for sublist in expanded_rows for item in sublist])

# Has to be a new variable, since expanded_campuses returns a Series.
expanded_campuses = df_CSF.apply(expand_campuses, axis = 1)
df_CSF = pd.DataFrame([item for sublist in expanded_campuses for item in sublist])

# Replace campus common name with BANNER Recognized name.
df_CSF['CAMPUS'] = df_CSF['CAMPUS'].replace({'BCC': 'FBC', 'LC': 'FLC', 'WC': 'FWC', 'OL': 'FCY'})


''' Functions to add required database information based off of existing data within the Course Specific Fees Dataset'''
# Function to return the Fee Type for SSADETL based off charges in the "FREQUENCY" column
def fee_type(freq):
    if freq in ['Per Course', 'Per Term']:
        return 'FLAT'
    else:
        return 'CRED'

# Matches detail codes for FRCC and CO Online
def detail_code(det):
    if det == 'Digital Content Fee':
        return Dig_content_fee # A392 - FALL, A393 - SPRING, 'A394' Summer
    else:
        return Not_DCF # A383 - FALL, A384 - SPRING , 'A385' Summer # Course Specific Fee

# Applying the function to a new column called "Fee Type" based off the values from "FREQUENCY"
df_CSF['FEE TYPE'] = df_CSF['FREQUENCY'].apply(fee_type)

# Applying the function to a new column called "DETAIL CODE" based off the values from "EXPLANATION" and "CAMPUS"
df_CSF['DETAIL CODE'] = df_CSF.apply(lambda row: detail_code(row['EXPLANATION']), axis=1)

# Sort values by SUBJECT then COURSE NUMBER
df_CSF = df_CSF.sort_values(by=['SUBJECT', 'COURSE NUMBER'], ascending = [True, True])

print("\nInformation about the transformed Course Specific Fees Dataset\n")
display(df_CSF.head())
print(f"\nColumns: {df_CSF.shape[1]}\nRows: {df_CSF.shape[0]}")

df_CSF.to_excel(OUTPUT_DIR / Cleaned_CSF, index=False)


Information about the transformed Course Specific Fees Dataset



,CAMPUS,SUBJECT,COURSE NUMBER,SECTION,COURSE NAME,FY26 FEE AMOUNT,FREQUENCY,EXPLANATION,UPDATED 8/4/25 NOTES,FEE TYPE,DETAIL CODE
0,ALL,ABM,ALL,ALL,NaN,9.05,Per Credit Hour,HIGH,NaN,CRED,A383
1,FWC,ACC,1001,0XX,Fundamentals of Accounting,141.17,Per Term,Digital Content Fee,NaN,FLAT,A392
1,FWC,ACC,1001,09X,Fundamentals of Accounting,141.17,Per Term,Digital Content Fee,NaN,FLAT,A392
1,FWC,ACC,1001,37X,Fundamentals of Accounting,141.17,Per Term,Digital Content Fee,NaN,FLAT,A392
1,FWC,ACC,1001,38X,Fundamentals of Accounting,141.17,Per Term,Digital Content Fee,NaN,FLAT,A392



Columns: 11
Rows: 1109


### Merging data for Final Output.

In [33]:
# Merging the two DataFrames together, with a indicator from where each duplicate column is from.
# Can only mergy by "SUBJECT". Cannot merge by "COURSE NUMBER" since there are a few subjects where ALL courses have a fee. (Like CSC)
df_RT = pd.merge(df_Banner_CFL, df_CSF, how='inner', on=['SUBJECT'], suffixes=('_CFL', '_CSF'))

# Added for depreciation warning
all_cols = list(df_RT.columns)

# Creating a new Column "UNCHANGED" to see if the fee amount has is the same as what is in Banner's Course Fee Listing
df_RT['UNCHANGED'] = df_RT[f'FY{FY} FEE AMOUNT'] == df_RT['AMOUNT']

# Function to seperate HIGH and MED Attributes two dateframes (Since they are attribute costs, not course costs)
def filter_out_high_med_attr(df):

    # Filtering out "HIGH" or "MED" in the "EXPLANATION" column
    df = df[~((df['EXPLANATION'].str.contains('HIGH|MED', case=False, na=False)) &
    # Filtering out where above criteria is met AND "FY FEE AMOUNT" == Specified Amt
        (df[f'FY{FY} FEE AMOUNT'] == FYFeeAmt))]

    return df

# Function to match the "MODIFIED_SECTION" to the "SECTION_CSF" column or both to "ALL"
#   Note* MODIFIED_SECTION is used because it is modified from SECTION_CFL to catch the first or second number and replace the rest with X(s)
def section_filter(row):
    section_match = (
    (row['MODIFIED_SECTION'] == row['SECTION_CSF']) or
    (row['MODIFIED_SECTION'] == 'ALL') or
    (row['SECTION_CSF'] == 'ALL')
                    )
    return section_match

# Function to match the "COURSE NUMBER_CFL" to the "COURSE NUMBER_CSF" column or both to "ALL"
def course_filter(row):

    course_match = (
    (row['COURSE NUMBER_CFL'] == row['COURSE NUMBER_CSF']) or
    (row['COURSE NUMBER_CFL'] == 'ALL') or
    (row['COURSE NUMBER_CSF'] == 'ALL')
                    )
    return course_match

# Function to filter out HS courses ('37X', '38X', '39X') when they match with "ALL" from the course_filter function
def dropHS_all(row):
    if row['MODIFIED_SECTION'] in ['37X', '38X', '39X'] and row['SECTION_CSF'] == 'ALL':
        return False
    return True

# Function to match the "CAMPUS_CFL" to the "CAMPUS_CSF" column Based off specific Campus codes for matching
def campus_filter(row):

    # Snippet to match Boulder Off-Site Campus with Boulder Off-Site and Boulder Campus
    if row['CAMPUS_CFL'] == 'FBO':
        campus_match = row['CAMPUS_CSF'] in ['FBO', 'FBC', 'ALL']
    # Snippet to match Westminster Off-Site Campus with Westminster Off-Site and Westminster Campus
    elif row['CAMPUS_CFL'] == 'FWO':
        campus_match = row['CAMPUS_CSF'] in ['FWO', 'FWC', 'ALL']
    # Snippet to match Fort Collins Off-Site campus with Fort Collins Off-Site and Fort Collins Campus
    elif row['CAMPUS_CFL'] == 'FLO':
        campus_match = row['CAMPUS_CSF'] in ['FLO', 'FLC', 'ALL']
    # Snippet to match Colorado Online Campus with Online and Colorado Online
    elif row['CAMPUS_CFL'] == 'FCY':
        campus_match = row['CAMPUS_CSF'] in ['FON', 'FCY', 'ALL'] # Added 'ALL' for testing
    # Snippet to match for "ALL" or the same Campus code
    else:
        campus_match = (
                (row['CAMPUS_CFL'] == row['CAMPUS_CSF']) or
                (row['CAMPUS_CFL'] == 'ALL') or
                (row['CAMPUS_CSF'] == 'ALL')
                        )
    return campus_match

# Function to filter out Sections that aren't 2XX, 3XX, 37X, or 38X if the ATTR is CONC
def hs_filter(df):
        # Exclude rows where ATTR is 'CONC' and SECTION is '2XX' or '3XX' or '30X'
    df = df[~((df['ATTR'] == "CONC") &
              (df['MODIFIED_SECTION'].str.match(r'^(2XX|3XX|7XX)$', na=False)))]

    return df

# Applying all the functions to the DataFrame to futher refine the results
df_RT = filter_out_high_med_attr(df_RT)
df_RT = df_RT[df_RT.apply(section_filter, axis=1)]
df_RT = df_RT[df_RT.apply(campus_filter, axis=1)]
df_RT = df_RT[df_RT.apply(course_filter, axis=1)]
df_RT = df_RT[df_RT.apply(dropHS_all, axis=1)]
df_RT = hs_filter(df_RT)

# Function to see if Fee Amounts have changed
def CRN_fee_changed(group):
    previous_fee = set(group['AMOUNT'].astype(float).round(2))
    current_fee = set(group[f'FY{FY} FEE AMOUNT'].astype(float).round(2))

    # Check Previous Fee Amount for Any Matching value in Current Fee Amount for the specific CRN
    unchanged = group['AMOUNT'].apply(lambda x: x in current_fee)

    # Identify any Current Fees that were added and not in previous_fee so it can be added
    new_fees = current_fee - previous_fee

    # If a new fee exists, mark at least ONE row as False for input
    if new_fees:
        index_to_modify = group.index[0] # Modify first row in the group
        unchanged.loc[index_to_modify] = False # Flag one row as False

    return unchanged

# Function to see if Detail Codes have changed
def CRN_DC_changed(group):
    prev_dc = set(group['DET CODE'].astype(str).str.strip().str.upper())
    current_dc = set(group['DETAIL CODE'].astype(str).str.strip().str.upper())

    # Any code from Previous appearing in Current Detail Code(s)
    unchanged = group['DET CODE'].apply(lambda x: x in current_dc)

        # Identify any Current Fees that were added and not in previous_fee so it can be added
    new_codes = current_dc - prev_dc

    # If a new fee exists, mark at least ONE row as False for input
    if new_codes:
        index_to_modify = group.index[0] # Modify first row in the group
        unchanged.loc[index_to_modify] = False # Flag one row as False

    return unchanged

# Function to see if Fee Types have changed
def CRN_FT_changed(group):
    prev_ft = set(group['FEE TYPE (OLD)'].astype(str).str.strip().str.upper())
    current_ft = set(group['FEE TYPE'].astype(str).str.strip().str.upper())

    # Any code from Previous appearing in Current Fee Type(s)
    unchanged = group['FEE TYPE (OLD)'].apply(lambda x: x in current_ft)

        # Identify any Current Fees that were added and not in previous_fee so it can be added
    new_fee_type = current_ft - prev_ft

    # If a new fee exists, mark at least ONE row as False for input
    if new_fee_type:
        index_to_modify = group.index[0] # Modify first row in the group
        unchanged.loc[index_to_modify] = False # Flag one row as False

    return unchanged

# Applying function to get the outcome of Fee Changes
df_RT['UNCHANGED'] = df_RT.groupby('CRN', group_keys=False)[all_cols].apply(CRN_fee_changed)

# Applying function to get the outcome of Detail Code Changes
df_RT.loc[df_RT['UNCHANGED'], 'UNCHANGED'] = df_RT.groupby('CRN', group_keys=False)[all_cols].apply(CRN_DC_changed)

# Applying function to get the outcome of Fee Type Changes
df_RT.loc[df_RT['UNCHANGED'], 'UNCHANGED'] = df_RT.groupby('CRN', group_keys=False)[all_cols].apply(CRN_FT_changed)

# Changing "UNCHANGED" from a bool to string
df_RT['UNCHANGED'] = df_RT['UNCHANGED'].astype(object)

mp_mask = df_RT['EXPLANATION'].str.contains("Malpractice Insurance", na=False)
df_RT.loc[mp_mask, 'UNCHANGED'] = 'MP'

# Applying a final change to have MP in "Unchanged Column" if Explanation has "Malpractice Insurance"
df_RT.loc[df_RT['EXPLANATION'].str.contains("Malpractice Insurance", na=False), "UNCHANGED"] = 'MP'

df_RT = df_RT.sort_values(by=['SUBJECT', 'COURSE NUMBER_CFL', 'CRN'], ascending = [True, True, True])

# Changing column order so related columns from the two datasets are near each other
column_order = ['SEMESTER', 'CRN', 'SUBJECT', 'COURSE NUMBER_CFL', 'COURSE NUMBER_CSF', 'SECTION_CFL', 'MODIFIED_SECTION', 'SECTION_CSF', 'CAMPUS_CFL', 'CAMPUS_CSF', 'ATTR', 'DET CODE', 'DETAIL CODE', 'AMOUNT', f'FY{FY} FEE AMOUNT', 'FEE TYPE (OLD)', 'FEE TYPE', 'COURSE NAME', 'FREQUENCY', 'EXPLANATION', 'UNCHANGED']
df_RT = df_RT[column_order]

# Getting a quick preview of what the data will look like, along with the count of columns and rows, before creating an excel file.
print("\nInformation about the transformed Rate Table Dataset\n")
display(df_RT.head())
print(f"\nColumns: {df_RT.shape[1]} \nRows: {df_RT.shape[0]}")


Information about the transformed Rate Table Dataset



,SEMESTER,CRN,SUBJECT,COURSE NUMBER_CFL,COURSE NUMBER_CSF,SECTION_CFL,MODIFIED_SECTION,SECTION_CSF,CAMPUS_CFL,CAMPUS_CSF,...,DET CODE,DETAIL CODE,AMOUNT,FY26 FEE AMOUNT,FEE TYPE (OLD),FEE TYPE,COURSE NAME,FREQUENCY,EXPLANATION,UNCHANGED
3,202620,53178,ACC,1001,1001,38A,38X,38X,FWO,FWC,...,A392,A392,141.17,141.17,FLAT,FLAT,Fundamentals of Accounting,Per Term,Digital Content Fee,True
130,202620,53527,ACC,1011,1011,604,6XX,6XX,FLC,FLC,...,A392,A392,92.31,92.31,FLAT,FLAT,Intro to Financial Accounting,Per Term,Digital Content Fee,True
241,202620,53528,ACC,1011,1011,602,6XX,6XX,FLC,FLC,...,A392,A392,92.31,92.31,FLAT,FLAT,Intro to Financial Accounting,Per Term,Digital Content Fee,True
352,202620,53529,ACC,1011,1011,603,6XX,6XX,FLC,FLC,...,A392,A392,92.31,92.31,FLAT,FLAT,Intro to Financial Accounting,Per Term,Digital Content Fee,True
463,202620,53530,ACC,1011,1011,601,6XX,6XX,FLC,FLC,...,A392,A392,92.31,92.31,FLAT,FLAT,Intro to Financial Accounting,Per Term,Digital Content Fee,True



Columns: 21 
Rows: 1931


### Finding Fees that need removed

In [34]:
''' Code to make a duplicate of df_RT but for fees that need to be removed'''

# Get the list of valid subjects from Course Specific Fees
valid_subjects = set(df_CSF['SUBJECT'].unique())

# Filter Banner rows where the subject does NOT exist in CSF
df_subject_not_in_CSF = df_Banner_CFL[~df_Banner_CFL['SUBJECT'].isin(valid_subjects)].copy()

# Add a column to indicate the removal reason
df_subject_not_in_CSF['REMOVAL_REASON'] = 'Subject not in CSF'

# Step 2: Flag course numbers in Banner that do not exist in CSF for the same subject

# Only consider Banner rows where SUBJECT exists in CSF
df_Banner_subject_match = df_Banner_CFL[df_Banner_CFL['SUBJECT'].isin(df_CSF['SUBJECT'])].copy()

# Build valid (SUBJECT, COURSE NUMBER) pairs from CSF
valid_subj_course = df_CSF[['SUBJECT', 'COURSE NUMBER']].drop_duplicates()
valid_subj_course_set = set([tuple(x) for x in valid_subj_course.values])

# Identify (SUBJECT, COURSE NUMBER) pairs in Banner that don't exist in CSF
df_Banner_subject_match['SUBJ_COURSE_PAIR'] = list(zip(df_Banner_subject_match['SUBJECT'], df_Banner_subject_match['COURSE NUMBER']))
mask_invalid_course = ~df_Banner_subject_match['SUBJ_COURSE_PAIR'].isin(valid_subj_course_set)

# Filter those rows and add reason
df_course_not_in_CSF = df_Banner_subject_match[mask_invalid_course].copy()
df_course_not_in_CSF['REMOVAL_REASON'] = 'Course Number not in CSF'

# Filter to rows where subject and course number are valid (already handled by prior steps)
df_Banner_course_match = df_Banner_CFL[
    df_Banner_CFL['SUBJECT'].isin(df_CSF['SUBJECT']) &
    df_Banner_CFL[['SUBJECT', 'COURSE NUMBER']].apply(tuple, axis=1).isin(
        df_CSF[['SUBJECT', 'COURSE NUMBER']].drop_duplicates().apply(tuple, axis=1)
    )
].copy()


# Normalize SECTIONs before anything else
df_CSF['SECTION'] = df_CSF['SECTION'].fillna('ALL').replace('', 'ALL').str.strip().str.upper()
df_Banner_CFL['SECTION'] = df_Banner_CFL['SECTION'].fillna('ALL').replace('', 'ALL').astype(str).str.strip().str.upper()

# Rebuild MODIFIED_SECTION after normalization
df_Banner_CFL['MODIFIED_SECTION'] = df_Banner_CFL['SECTION'].apply(
    lambda s: s if s == 'ALL' else (s[:2] + 'X' if s[:2] in ['37', '38', '39', '27', '28', '78'] else s[0] + 'XX')
)

# Filter rows with matching SUBJECT and COURSE NUMBER only
df_Banner_course_match = df_Banner_CFL[
    df_Banner_CFL['SUBJECT'].isin(df_CSF['SUBJECT']) &
    df_Banner_CFL[['SUBJECT', 'COURSE NUMBER']].apply(tuple, axis=1).isin(
        df_CSF[['SUBJECT', 'COURSE NUMBER']].drop_duplicates().apply(tuple, axis=1)
    )
].copy()

# Build valid exact (SUBJECT, COURSE NUMBER, SECTION) keys
valid_sections = df_CSF[['SUBJECT', 'COURSE NUMBER', 'SECTION']].drop_duplicates()
valid_section_keys = set([tuple(x) for x in valid_sections.values])

# Build wildcard keys where SECTION == 'ALL'
valid_wildcard_keys = set([
    (row['SUBJECT'], row['COURSE NUMBER'], 'ALL')
    for _, row in df_CSF.iterrows()
])

# Build SECTION_KEY for comparison using MODIFIED_SECTION
df_Banner_course_match['SECTION_KEY'] = list(zip(
    df_Banner_course_match['SUBJECT'],
    df_Banner_course_match['COURSE NUMBER'],
    df_Banner_course_match['MODIFIED_SECTION']
))

# Final allowed keys = exact matches + wildcards
all_valid_keys = valid_section_keys.union(valid_wildcard_keys)

def section_key_matches(key, valid_keys):
    subj, course, section = key
    is_hs = section in ['37X', '38X', '39X', '27X', '28X', '78X']

    # Exact match is always OK
    if (subj, course, section) in valid_keys:
        return True

    # HS sections must NOT be allowed to match 'ALL'
    if is_hs and (subj, course, 'ALL') in valid_keys:
        return False

    # Otherwise, allow 'ALL' fallback
    return (subj, course, 'ALL') in valid_keys

df_Banner_course_match['SECTION_MATCH'] = df_Banner_course_match['SECTION_KEY'].apply(
    lambda key: section_key_matches(key, valid_section_keys)
)

# Now flag anything that failed both exact and wildcard
df_section_not_in_CSF = df_Banner_course_match[~df_Banner_course_match['SECTION_MATCH']].copy()
df_section_not_in_CSF['REMOVAL_REASON'] = 'Section not found for Subject & Course Number in CSF'

#Concat

# Combine all flagged removal dataframes
fees_to_remove = pd.concat([
    df_subject_not_in_CSF,
    df_course_not_in_CSF,
    df_section_not_in_CSF
], ignore_index=True)

# Ensure AMOUNT is numeric for reliable filtering
fees_to_remove['AMOUNT'] = pd.to_numeric(fees_to_remove['AMOUNT'], errors='coerce')

# Filter out rows where AMOUNT is NaN or 0
fees_to_remove = fees_to_remove[fees_to_remove['AMOUNT'].notna() & (fees_to_remove['AMOUNT'] != 0)]
fees_to_remove.drop(columns=['SUBJ_COURSE_PAIR', 'SECTION_KEY', 'SECTION_MATCH'], errors='ignore', inplace=True)

fees_to_remove.sort_values(
    by=['SUBJECT', 'COURSE NUMBER', 'SECTION', 'CRN'],
    ascending=[True, True, True, True],
    inplace=True,
    key=lambda col: col.astype(str).str.upper() if col.name in ['SUBJECT', 'SECTION'] else col
)

## Final output

In [35]:
with pd.ExcelWriter(OUTPUT_DIR / Rate_Table, engine='openpyxl') as writer:
    df_RT.to_excel(writer, sheet_name='Rate Table', index=False)
    fees_to_remove.to_excel(writer, sheet_name='Fee Removal', index=False)